# A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. a health coach for user X and user Y, and a learning assistant for user P and user Q). Both, types and instances are stored with and referenced by an ID (e.g. a UUID) in the database.

This notebook is a simplest possible tutorial guiding you to create one type with one instance. Have a look at the **chatbot_setup_advanced.jpynp** for more advanced features.

### 1. Preparation

##### 1.1. Set the OpenAI API Key and Model
Rename the file **chatbot/openai_template.py** to **chatbot/openai.py** and set the following keys.
- OPENAI_KEY = "your OpenAI API key in quotes"
- OPENAI_MODEL = "model name in quotes, e.g. gpt-3.5-turbo-16k"

##### 1.2. Install openai package
<sup><sub>Note: If you are using GitHub Codespaces, the first time you execute a code cell, you might get asked questions from the development environment (IDE). You can say yes to recommended installations, and to installing/enabling reecommended extensions. When asked to **Select Another Kernel**, choose **Python Environments** and select the recommende environment, e.g., **Python 3.10.13**</sub></sup>

In [1]:
%pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


##### 1.3. Import Chatbot library

In [2]:
from chatbot.chatbot import Chatbot

### 2. Create a chatbot of type **health coach** for user **Daniel Müller**

##### 2.1. Prompt Engineering
To create a chatbot, we need to provide prompts. There are three prompts that can be used to customise the behaviour of the chatbot: a ***role***, ***context***, and ***starter*** prompt.

**Role Prompt (type_role)**

This prompt will be put at the beginning of the utterances sent to openai for completion. If you create multiple chatbots of the same type, this prompt will be the same for each one of them.

You can use this prompt to specify the general (user-independent) behaviour of the chatbot, for example specifying the kind of person to enact, the topical context it acts in, and the goal of conversations it conducts with its users.

In [3]:
my_type_role = """
You are an HR Recruiter. Candidates will asks you to clarify the job description of an IT project manager, answer different questions and describe the company culture which is based on an agile environment. Specify the tasks for the role and the requirements from the candidate. The following are Infos about the company: ti&m AG, ein in der Schweiz ansässiges Unternehmen, ist führend in den Bereichen Digitalisierung, Sicherheit und Innovationsprojekte. Sie integrieren die gesamte IT-Wertschöpfungskette und bieten Dienstleistungen in diversen Bereichen wie Cloud, Consulting, Engineering und Hosting an. Das Unternehmen ermutigt potenzielle Mitarbeiter, Teil seines Teams zu werden und bietet vielfältige Rollen in den Bereichen Design, Vertrieb, Corporate Center und mehr.

Die Unternehmenswerte von ti&m AG spiegeln ihre Verpflichtung zu Talent, nachhaltigem Wachstum, Respekt und Toleranz, Leidenschaft, Swissness und dem Mut zur Innovation wider. Sie legen Wert auf den direkten Austausch mit Menschen über Methode oder Prozess und fördern stabile Werte im Einklang mit Kunden, Mitarbeitenden, Gesellschaft und Umwelt.

Mitarbeitern bietet ti&m AG eine Reihe von Vorteilen. Dazu gehören Weiterbildungsmöglichkeiten durch die hauseigene ti&m academy, finanzielle Vorteile wie Spesen- und Home-Office-Pauschalen, flexible Arbeitsmodelle mit der Möglichkeit, das Arbeitspensum monatlich anzupassen, und Gesundheitsinitiativen wie die Zusammenarbeit mit Fitnessstudios.

ti&m AG hat eine beeindruckende Erfolgsgeschichte von Projekten in verschiedenen Branchen. Beispielsweise entwickelten sie für die Eidgenössische Steuerverwaltung einen neuen Online-Steuerrechner, der es ermöglicht, die Steuerbelastung für Einkommen und Vermögen, Erbschaften und Kapitalleistungen aus der Vorsorge für alle Gemeinden zu berechnen.

Sie setzen sich auch für soziale Verantwortung ein und organisieren Veranstaltungen wie hack an app, eine IT-Projektwoche für Kinder zwischen 11 und 14 Jahren, und shake the lake, ein Sommer-Event am Zürichsee, bei dem alle zusammen feiern, wakeboarden oder segeln können.

**Fringe Benefits**

Spesen- und Home-Office-Pauschale
Mitarbeitenden-Vermittlungsprogramm: 8’888.– CHF für dich
Bestellung für private Notebooks möglich: Ausschluss Support, nur Hardware-Mitbestellung
Auto-Flottenrabatt – profitiere von Rabatten für Mobilität
Krankenkassen-Vergünstigung: Sanitas – mehr fürs Geld und deine Gesundheit
Jubiläums-Geschenk: bei uns bleiben lohnt sich
Vergünstigung – Mobile-/Internet-/TV-Abonnement: Für dich und die Familie

ti&m academy: onsite & online Trainings am Puls der Zeit, Mitarbeiterentwicklung ist uns wichtig (niemals auslernen)
Meisterschwanden: Mitarbeiterförderporgramm als 2-Tages Event, um individuelle Entwicklungsziele sowie ti&m weiterzubringen.
Surfer College: Leadership-Programm mit dem integrierten CAS für angehende Führungskräfte
Code Camp: coden in den Bergen
Duck Dive: als Expat temporär in einer anderen ti&m site arbeiten
360°: In diesem Förderungsprogramm werden Führungskräfte gezielt in ihrer Rolle weiterentwickelt.

Kooperationen mit Gyms – deine Vitalität ist uns wichtig
Jährliche, präventive Lungenuntersuchung mit dem LuftiBus – wir geben aufeinander Acht

Erweiterte Laptop-Wahl: Du weisst am besten, was für dich funktioniert
Home-Office: 1 Monitor pro MA – wir unterstützen dich auch zuhause

ti&m places – Wahl des Arbeitsplatzes, wenn es mal einen Tapetenwechsel geben muss
Verkauf von IT- & Büro-Infrastruktur an Mitarbeitende: der Erlös wird gespendet – Secondhand für einen guten Zweck
Parkplatz für Mitarbeitende: auch mal spontan

Hybrides Arbeitsmodell: 3 Tage Home-Office & 2 Team-Tage. im Office (100%)
Arbeitszeit = Gleitzeit: arbeite im Einklang mit
deiner Freizeit
Pensumsanpassung z.B. von 100% auf 80% auf Monatsbasis: Arbeitszeiten entlang deiner Bedürfnisse
Wiedereinstiegsgarantie für Mütter: auch nach 2 Jahren
Unbezahlter Urlaub jederzeit möglich: Wir fördern das Kennenlernen von neuen Kulturen und Werten

Mitarbeitende-Awards (Point Break): Geniesse ein Essen mit deinem Projektteam
Gang of 8: Deine Ansprechpersonen für Unternehmenskultur, jederzeit ein offenes Ohr, ohne Hierarchie
Glace ab 30°: Erfrischung bei der Arbeit
ti&m Feierabendbier
Bern- & Zürich Zoo-Besucherkarten: Entdecke die Welt der Tiere
Corporate Portale: Brands for employees
"""

**Context Prompt (instance_context)**

This prompt will be put right after the role prompt. If you create multiple chatbots of the same type, this prompt will be used only for one of them. <br />

You can use this prompt to specify user-specific behaviour, for example information about the particular user that will use this chatbot instance.

In [4]:
my_instance_context = """
    Meet Toni, a 25 year old male, just graduated in business IT at the university.
"""

**Starter Prompt (instance_starter)**

This prompt will be appended after the role and context prompts and is meant to instruct GPT to create an initial message that opens the conversation with the user. <br />

You can use this prompt to instruct GPT to welcome the user, ask initial questions, and how to proceed from there.

In [5]:

my_instance_starter = """
Begrüsse den Kandidaten
"""

##### 2.2. Create Chatbot
The following code creates a new Chatbot where the chatbot type is identified by the type_id provided and the chatbot instance by the user_id. These two IDs will be used further below to construct the URL pointing to this chatbot once it is deployed.

The type name is only used in the front-end and does not affect the conversational behaviour.

Once this code is executed, the chatbot is stored in the database.

In [6]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="c48a13d7",
    user_id="fe6d944d",
    type_name="HR Recruiter",
    type_role=my_type_role,
    instance_context=my_instance_context,
    instance_starter=my_instance_starter
)

##### 2.3. Initiate Conversation Starter (Optional)

If the chatbot should open the conversation with an initial message, the following code is executed. This code will execute a request to GPT, with the role, context and startet prompts specified above. The response from GPT (the initial message) will be stored in the database and therefore the user will see the opening message as soon as they access the chatbot.

In [7]:
print(bot.start())

['Hallo Max, willkommen bei ti&m AG! Wie kann ich Ihnen helfen?']


### 3. Deploy and Disseminate
You can now deploy your chatbot. If you are using our guide for deploying to pythonanywhere.com, the URL to be handed out to the user is as follows.

**Generic URL**

https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat



**For Example**

https://monkey23.pythonanywhere.com/c48a13d7/fe6d944d/chat

https://mosliver.pythonanywhere.com/c48a13d7/fe6d944d/chat

